# GDELT 이벤트 → 자연어 스토리 변환기

GDELT의 구조화된 데이터를 읽기 쉬운 자연어 문장으로 변환하여 지정학적 이벤트를 스토리텔링 형태로 분석

In [1]:
import sys

sys.path.append("/app")

from src.utils.spark_builder import get_spark_session
from pyspark.sql import functions as F
from pyspark.sql.types import StringType

# Spark 세션 생성
spark = get_spark_session("GDELT_Story_Generator", "spark://spark-master:7077")
print("✅ Spark 세션 생성 완료")

25/09/15 12:45:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/15 12:45:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


✅ Spark 세션 생성 완료


In [2]:
# Silver 데이터 로드
joined_path = "s3a://warehouse/silver/test_gdelt_joined/"
final_silver_df = spark.read.format("delta").load(joined_path)

print("✅ 데이터 로드 완료")
print(f"총 레코드 수: {final_silver_df.count():,}")
print(f"컬럼 수: {len(final_silver_df.columns)}")

25/09/15 12:45:46 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


✅ 데이터 로드 완료


25/09/15 12:45:49 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


총 레코드 수: 12,241
컬럼 수: 85


In [3]:
# 1. 경로 추가
import sys

sys.path.append("/app/notebooks")

# 2. 함수들 import
from test import create_simple_story, create_rich_story, extract_rich_themes

print("✅ 스토리 생성 함수 import 완료!")

# 실제 데이터로 테스트
random_sample = final_silver_df.orderBy(F.rand()).take(10)

print("\n🔸 스토리 생성:")
for i, row in enumerate(random_sample, 1):
    row_dict = row.asDict()
    print("🔸 데이터 확인:")
    simple = create_simple_story(row_dict)
    rich = create_rich_story(row_dict)

    print(f"{i}. 📖 간단: {simple}")
    print(f"   🌟 풍부: {rich}")

🔸 스토리 생성 함수 정의 시작...
✅ 함수 정의 완료!
📋 사용 가능한 함수:
   - create_simple_story(row): 간단한 스토리
   - create_rich_story(row): 풍부한 스토리
   - test_story_generation(): 실제 데이터로 테스트
   - search_event_by_id(event_id): 특정 이벤트 검색
✅ 스토리 생성 함수 import 완료!



🔸 스토리 생성:
🔸 데이터 확인:
1. 📖 간단: POLICE OFFICER이(가) Kathmandu에서 AUTHORITIES에게 193 액션을 수행했다 (performed action 193) (부정적으로 (negatively))
   🌟 풍부: 🔥 주요 사건 (MAJOR EVENT): Khadga Prasad Oli,2276 from POLICE OFFICER이(가) Kathmandu에서 AUTHORITIES에게 강한 부정적 감정으로 (with strong negative sentiment) 193 액션을 취했다 (took action 193) 세금 (taxation) 금융 활동 (financial activities) woman, 652 ict applications, 662 social media에 관해 (concerning 세금 (taxation) 금융 활동 (financial activities) woman, 652 ict applications, 662 social media) Youtube,1337, Supreme Court,134와 관련하여 (involving Youtube,1337, Supreme Court,134) 51,people dead,300, 10000,of thousands taking,1328 규모로 (worth 51,people dead,300, 10000,of thousands taking,1328) (20개 기사에서 보도됨, californiatelegraph.com 포함 / reported across 20 articles including californiatelegraph.com)
🔸 데이터 확인:
2. 📖 간단: UNIVERSITY이(가) unknown location에서 UNIVERSITY에게 111 액션을 수행했다 (performed action 111) (부정적으로 (negatively))
   🌟 풍부: Ahman Andrews,1113 from UNIVERSITY이(가) unknown location에서 